In [1]:
import pandas as pd
import numpy as np
import wrangle
import seaborn as sns
import warnings
import pydataset
warnings.filterwarnings("ignore")

from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectKBest, f_regression

Do your work for this exercise in a jupyter notebook named feature_engineering within the regression-exercises repo. Add, commit, and push your work.

# 1. Load the tips dataset.

 - Create a column named tip_percentage. This should be the tip amount divided by the total bill. 

In [2]:
tips = sns.load_dataset('tips')
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [3]:
tips['tip_percentage'] = tips.tip / tips.total_bill
tips.head()

,total_bill,tip,sex,smoker,day,time,size,tip_percentage
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808


 - Create a column named price_per_person. This should be the total bill divided by the party size.
 

In [4]:
tips['price_per_person'] = tips.total_bill / tips.size
tips.head()

,total_bill,tip,sex,smoker,day,time,size,tip_percentage,price_per_person
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447,0.008704
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542,0.005297
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587,0.010763
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780,0.012131
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808,0.012597


 - Before using any of the methods discussed in the lesson, which features do you think would be most important for    predicting the tip amount? The tip percentage?

In [5]:
# I'm guessing total bill is most important for predicting tip amount and size for tip percentage

 - Use all the other numeric features to predict tip amount. Use select k best and recursive feature elimination to    select the top 2 features. What are they?

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
def train_validate_test(df, target):
    '''
    this function takes in a dataframe and splits it into 3 samples, 
    a test, which is 20% of the entire dataframe, 
    a validate, which is 24% of the entire dataframe,
    and a train, which is 56% of the entire dataframe. 
    It then splits each of the 3 samples into a dataframe with independent variables
    and a series with the dependent, or target variable. 
    The function returns 3 dataframes and 3 series:
    X_train (df) & y_train (series), X_validate & y_validate, X_test & y_test. 
    '''
    # split df into test (20%) and train_validate (80%)
    train_validate, test = train_test_split(df, test_size=.2, random_state=123)

    # split train_validate off into train (70% of 80% = 56%) and validate (30% of 80% = 24%)
    train, validate = train_test_split(train_validate, test_size=.3, random_state=123)

        
    # split train into X (dataframe, drop target) & y (series, keep target only)
    X_train = train.drop(columns=[target])
    y_train = train[target]
    
    # split validate into X (dataframe, drop target) & y (series, keep target only)
    X_validate = validate.drop(columns=[target])
    y_validate = validate[target]
    
    # split test into X (dataframe, drop target) & y (series, keep target only)
    X_test = test.drop(columns=[target])
    y_test = test[target]
    
    return X_train, y_train, X_validate, y_validate, X_test, y_test

In [7]:
X_train, y_train, X_validate, y_validate, X_test, y_test = train_validate_test(tips, 'tip' )
X_train.head()

,total_bill,sex,smoker,day,time,size,tip_percentage,price_per_person
18,16.97,Female,No,Sun,Dinner,3,0.206246,0.008694
172,7.25,Male,Yes,Sun,Dinner,2,0.710345,0.003714
118,12.43,Female,No,Thur,Lunch,2,0.144811,0.006368
28,21.70,Male,No,Sat,Dinner,2,0.198157,0.011117
237,32.83,Male,Yes,Sat,Dinner,2,0.035638,0.016819


In [8]:
object_cols = ['sex','smoker','day','time']
X_train_encoded = pd.get_dummies(X_train, columns = object_cols, drop_first=True)
X_train_encoded

,total_bill,size,tip_percentage,price_per_person,sex_Female,smoker_No,day_Fri,day_Sat,day_Sun,time_Dinner
18,16.97,3,0.206246,0.008694,1,1,0,0,1,1
172,7.25,2,0.710345,0.003714,0,0,0,0,1,1
118,12.43,2,0.144811,0.006368,1,1,0,0,0,0
28,21.70,2,0.198157,0.011117,0,1,0,1,0,1
237,32.83,2,0.035638,0.016819,0,0,0,1,0,1
207,38.73,4,0.077459,0.019841,0,0,0,1,0,1
183,23.17,4,0.280535,0.011870,0,0,0,0,1,1
60,20.29,2,0.158206,0.010394,0,0,0,1,0,1
41,17.46,2,0.145475,0.008945,0,1,0,0,1,1
160,21.50,4,0.162791,0.011014,0,1,0,0,1,1


In [9]:
X_train.drop(columns=object_cols,inplace=True)
X_train

,total_bill,size,tip_percentage,price_per_person
18,16.97,3,0.206246,0.008694
172,7.25,2,0.710345,0.003714
118,12.43,2,0.144811,0.006368
28,21.70,2,0.198157,0.011117
237,32.83,2,0.035638,0.016819
207,38.73,4,0.077459,0.019841
183,23.17,4,0.280535,0.011870
60,20.29,2,0.158206,0.010394
41,17.46,2,0.145475,0.008945
160,21.50,4,0.162791,0.011014


In [10]:
f_selector = SelectKBest(f_regression, k=2)
f_selector.fit(X_train, y_train)
feature_mask = f_selector.get_support()

In [11]:
feature_mask

array([ True, False, False,  True])

In [12]:
f_feature = X_train.iloc[:,feature_mask].columns.tolist()
f_feature

['total_bill', 'price_per_person']

In [13]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

lm = LinearRegression()
rfe = RFE(lm,2)
rfe.fit(X_train, y_train)
feature_mask_rfe = rfe.support_
rfe_feature = X_train.iloc[:,feature_mask].columns.tolist()

In [14]:
rfe_feature

['total_bill', 'price_per_person']

 - Use all the other numeric features to predict tip percentage. Use select k best and recursive feature elimination    to select the top 2 features. What are they?

In [15]:
X_train, y_train, X_validate, y_validate, X_test, y_test = train_validate_test(tips, 'tip_percentage' )
X_train.drop(columns=object_cols,inplace=True)

In [16]:
f_selector = SelectKBest(f_regression, k=2)
f_selector.fit(X_train, y_train)
feature_mask = f_selector.get_support()

f_feature = X_train.iloc[:,feature_mask].columns.tolist()
f_feature

['total_bill', 'tip']

In [17]:
lm = LinearRegression()
rfe = RFE(lm,2)
rfe.fit(X_train, y_train)
feature_mask_rfe = rfe.support_
rfe_feature = X_train.iloc[:,feature_mask].columns.tolist()
rfe_feature

['total_bill', 'tip']

 - Why do you think select k best and recursive feature elimination might give different answers for the top            features? Does this change as you change the number of features your are selecting?
 

In [18]:
# features aren't completely independent of eachother so the more features added the different their rankings in relation to eachother

# 2. Write a function named select_kbest that takes in the predictors (X), the target (y), and the number of features to select (k) and returns the names of the top k selected features based on the SelectKBest class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [19]:
def select_kbest(X, y, k):
    f_selector = SelectKBest(f_regression, k)
    f_selector.fit(X, y)
    feature_mask = f_selector.get_support()

    f_feature = X.iloc[:,feature_mask].columns.tolist()
    return f_feature

In [20]:
select_kbest(X_train, y_train, 2)

['total_bill', 'tip']

# 3. Write a function named rfe that takes in the predictors, the target, and the number of features to select. It should return the top k features based on the RFE class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [21]:
def rfe(X,y,k):
    lm = LinearRegression()
    rfe = RFE(lm,k)
    rfe.fit(X, y)
    feature_mask_rfe = rfe.support_
    rfe_feature = X.iloc[:,feature_mask_rfe].columns.tolist()
    return rfe_feature

In [22]:
rfe(X_train, y_train, 2)

['total_bill', 'tip']

# 4. Load the swiss dataset and use all the other features to predict Fertility. Find the top 3 features using both select k best and recursive feature elimination (use the functions you just built to help you out).

In [23]:
swiss = pydataset.data('swiss')
swiss

,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality
Courtelary,80.2,17.0,15,12,9.96,22.2
Delemont,83.1,45.1,6,9,84.84,22.2
Franches-Mnt,92.5,39.7,5,5,93.40,20.2
Moutier,85.8,36.5,12,7,33.77,20.3
Neuveville,76.9,43.5,17,15,5.16,20.6
Porrentruy,76.1,35.3,9,7,90.57,26.6
Broye,83.8,70.2,16,7,92.85,23.6
Glane,92.4,67.8,14,8,97.16,24.9
Gruyere,82.4,53.3,12,7,97.67,21.0
Sarine,82.9,45.2,16,13,91.38,24.4


In [24]:
X_train, y_train, X_validate, y_validate, X_test, y_test = train_validate_test(swiss, 'Fertility' )

In [25]:
select_kbest(X_train,y_train,3)

['Examination', 'Catholic', 'Infant.Mortality']

In [26]:
rfe(X_train,y_train,3)

['Agriculture', 'Examination', 'Infant.Mortality']